# Lyrics Analysis

In [1]:
lyrics_directory = "Lyrics_Data/"
tracks_csv = "tracks_group.csv"
stopwords_file = "stopwords.txt"

In [2]:
target_group = "itzy"
kpop_group = set(['kpop', 'itzy', 'seventeen', 'bts', 'twice'])
global_group = set(['global', 'billboard'])

target_and_kpop = kpop_group.union(set([target_group]))
target_and_global = global_group.union(set([target_group]))

## Import data

In [3]:
import pandas as pd

In [4]:
df = pd.read_csv(tracks_csv)

df.head(3)

,track_name,track_id,grouping
0,Ice Cream (with Selena Gomez),2J4P46vCFm1rPkNkp9pZWX,kpop
1,Dynamite,0v1x6rN6JHRapa03JElljE,kpop
2,THE BADDEST,2V4Fx72svQRxrFvNT1eq5f,kpop


In [5]:
df.shape

(4618, 3)

In [6]:
df["grouping"].unique()

array(['kpop', 'itzy', 'seventeen', 'bts', 'twice', 'global', 'billboard'],
      dtype=object)

In [7]:
def get_lyrics(track_id):
    try:
        lyrics = ""
        with open(lyrics_directory + track_id + ".txt", "r") as f:
            lyrics = f.read()
            
        return lyrics            
    except:
        return None
    
df["lyrics"] = df["track_id"].apply(get_lyrics)

df.shape

(4618, 4)

In [8]:
# stop words
# https://www.kaggle.com/datasets/rowhitswami/stopwords/
stopwords = set()
with open(stopwords_file, "r") as f:
    stopwords = set(f.read().split("\n"))
    
stopwords

{',',
 '>',
 '?',
 'Just',
 'Yes',
 'a',
 'abaft',
 'abafter',
 'abaftest',
 'about',
 'abouter',
 'aboutest',
 'above',
 'abover',
 'abovest',
 'accordingly',
 'aer',
 'aest',
 'afore',
 'after',
 'afterer',
 'afterest',
 'afterward',
 'afterwards',
 'again',
 'against',
 'aid',
 'ain',
 'albeit',
 'all',
 'aller',
 'allest',
 'alls',
 'allyou',
 'almost',
 'along',
 'alongside',
 'already',
 'also',
 'although',
 'always',
 'amid',
 'amidst',
 'among',
 'amongst',
 'an',
 'and',
 'andor',
 'anear',
 'anent',
 'another',
 'any',
 'anybody',
 'anyhow',
 'anyone',
 'anything',
 'anywhere',
 'apart',
 'aparter',
 'apartest',
 'appear',
 'appeared',
 'appearing',
 'appears',
 'appropriate',
 'appropriated',
 'appropriater',
 'appropriates',
 'appropriatest',
 'appropriating',
 'are',
 'ares',
 'around',
 'as',
 'ases',
 'aside',
 'asides',
 'aslant',
 'astraddle',
 'astraddler',
 'astraddlest',
 'astride',
 'astrider',
 'astridest',
 'at',
 'athwart',
 'atop',
 'atween',
 'aught',
 'aught

## Analyze

In [9]:
def is_stopword(word):
    word = word.lower()
    
    if len(word) <= 1:
        return True
    
    # in the list
    if word in stopwords:
        return True
    
    # is Korean
    if ord("가") <= ord(word[0]) <= ord("힣"):
        return True
    
    return False

### Characteristic Keyword
Using TF-IDF

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer

#### Among global

In [11]:
df_looking = df[df["grouping"].isin(target_and_global)].reset_index(drop=True)

In [12]:
vectorizer = TfidfVectorizer(stop_words=list(stopwords), max_features=1000, min_df=10)

x = vectorizer.fit_transform(df_looking["lyrics"].fillna("")).toarray()

df_tfidf = pd.DataFrame(x, columns=vectorizer.get_feature_names_out()).T

c:\Users\Hiyoko\.conda\envs\data\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['vis', 'viser', 'visest', 'yes'] not in stop_words.
  warnings.warn(


In [13]:
threshold = 0.001
counter = {}
summer = {}

for cnt in range(len(df_looking)):
    if df_looking["grouping"][cnt] != target_group:
        continue
    
    tfidf_series = df_tfidf[cnt]
    
    keywords = tfidf_series[tfidf_series > threshold].index.tolist()
    for keyword in keywords:
        counter[keyword] = counter.get(keyword, 0) + 1
        summer[keyword] = summer.get(keyword, 0) + float(tfidf_series[keyword])

In [14]:
def sort_dictionary(dictionary):
    return [(k, dictionary[k]) for k in sorted(dictionary, key=dictionary.get, reverse=True)]

In [15]:
tfidf_counter_global = sort_dictionary(counter)
tfidf_counter_global

[('sorry', 23),
 ('love', 22),
 ('mind', 22),
 ('yes', 21),
 ('let', 20),
 ('say', 19),
 ('tell', 14),
 ('feel', 14),
 ('baby', 13),
 ('down', 13),
 ('heart', 13),
 ('keep', 13),
 ('take', 12),
 ('care', 12),
 ('back', 12),
 ('bad', 11),
 ('eyes', 11),
 ('matter', 11),
 ('crazy', 11),
 ('think', 11),
 ('show', 10),
 ('control', 10),
 ('need', 10),
 ('room', 10),
 ('ah', 10),
 ('am', 9),
 ('turn', 9),
 ('put', 9),
 ('little', 9),
 ('forget', 9),
 ('way', 9),
 ('home', 9),
 ('only', 8),
 ('hear', 8),
 ('coming', 8),
 ('girl', 8),
 ('game', 8),
 ('doesn', 7),
 ('face', 7),
 ('honey', 7),
 ('ready', 7),
 ('break', 7),
 ('dance', 7),
 ('worry', 7),
 ('okay', 7),
 ('stay', 7),
 ('follow', 7),
 ('play', 7),
 ('head', 7),
 ('high', 7),
 ('hit', 7),
 ('light', 7),
 ('world', 7),
 ('cuz', 6),
 ('give', 6),
 ('child', 6),
 ('fight', 6),
 ('kill', 6),
 ('feeling', 6),
 ('uh', 6),
 ('boy', 6),
 ('didn', 6),
 ('start', 6),
 ('still', 6),
 ('top', 6),
 ('new', 6),
 ('free', 6),
 ('end', 6),
 ('lost',

In [16]:
tfidf_summer_global = sort_dictionary(summer)
tfidf_summer_global

[('love', 3.2111590137486092),
 ('sorry', 3.1624247485449186),
 ('trust', 2.3862535651729093),
 ('hot', 2.352824882534737),
 ('woo', 2.1142086197441174),
 ('mind', 2.1046574503957345),
 ('put', 2.020884450575801),
 ('flower', 1.980310452818118),
 ('em', 1.9626333700933452),
 ('shy', 1.9226858042564654),
 ('ready', 1.9188426805354042),
 ('worry', 1.837092103431801),
 ('yes', 1.8106564463544794),
 ('business', 1.8082325752768063),
 ('fall', 1.8011038824731416),
 ('heart', 1.7928345825331622),
 ('care', 1.7165829596424889),
 ('chi', 1.7042387770576761),
 ('keep', 1.6902832699626618),
 ('taking', 1.608198525916642),
 ('matter', 1.5818529324911066),
 ('home', 1.5356698179913768),
 ('finally', 1.5236500520493004),
 ('crazy', 1.510161642126581),
 ('think', 1.505804405173682),
 ('tomorrow', 1.4769442397116785),
 ('ah', 1.4542463159967527),
 ('words', 1.4389624091552111),
 ('control', 1.411335330255775),
 ('hit', 1.366162842085624),
 ('break', 1.3357711651378272),
 ('stronger', 1.28558964433042

#### Among KPOP

In [17]:
df_looking = df[df["grouping"].isin(target_and_kpop)].reset_index(drop=True)

In [18]:
vectorizer = TfidfVectorizer(stop_words=list(stopwords), max_features=1000, min_df=10)

x = vectorizer.fit_transform(df_looking["lyrics"].fillna("")).toarray()

df_tfidf = pd.DataFrame(x, columns=vectorizer.get_feature_names_out()).T

c:\Users\Hiyoko\.conda\envs\data\lib\site-packages\sklearn\feature_extraction\text.py:408: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['vis', 'viser', 'visest', 'yes'] not in stop_words.
  warnings.warn(


In [19]:
threshold = 0.001
counter = {}
summer = {}

for cnt in range(len(df_looking)):
    if df_looking["grouping"][cnt] != target_group:
        continue
    
    tfidf_series = df_tfidf[cnt]
    
    keywords = tfidf_series[tfidf_series > threshold].index.tolist()
    for keyword in keywords:
        counter[keyword] = counter.get(keyword, 0) + 1
        summer[keyword] = summer.get(keyword, 0) + float(tfidf_series[keyword])

In [20]:
tfidf_counter_kpop = sort_dictionary(counter)
tfidf_counter_kpop

[('sorry', 23),
 ('love', 22),
 ('mind', 22),
 ('yes', 21),
 ('let', 20),
 ('say', 19),
 ('tell', 14),
 ('feel', 14),
 ('baby', 13),
 ('down', 13),
 ('heart', 13),
 ('keep', 13),
 ('take', 12),
 ('care', 12),
 ('back', 12),
 ('bad', 11),
 ('eyes', 11),
 ('matter', 11),
 ('crazy', 11),
 ('think', 11),
 ('show', 10),
 ('control', 10),
 ('need', 10),
 ('room', 10),
 ('ah', 10),
 ('am', 9),
 ('turn', 9),
 ('put', 9),
 ('little', 9),
 ('forget', 9),
 ('way', 9),
 ('home', 9),
 ('only', 8),
 ('hear', 8),
 ('coming', 8),
 ('girl', 8),
 ('game', 8),
 ('doesn', 7),
 ('face', 7),
 ('honey', 7),
 ('ready', 7),
 ('한국', 7),
 ('break', 7),
 ('dance', 7),
 ('worry', 7),
 ('okay', 7),
 ('stay', 7),
 ('follow', 7),
 ('play', 7),
 ('head', 7),
 ('high', 7),
 ('informazioni', 7),
 ('hit', 7),
 ('light', 7),
 ('world', 7),
 ('cuz', 6),
 ('give', 6),
 ('territory', 6),
 ('child', 6),
 ('fight', 6),
 ('kill', 6),
 ('feeling', 6),
 ('uh', 6),
 ('boy', 6),
 ('didn', 6),
 ('start', 6),
 ('still', 6),
 ('top', 

In [21]:
tfidf_summer_kpop = sort_dictionary(summer)
tfidf_summer_kpop

[('love', 3.7295123013654305),
 ('put', 2.4826616351462447),
 ('trust', 2.4324877675268377),
 ('hot', 2.3126303347836705),
 ('keep', 2.0253074558606508),
 ('woo', 1.9559731310647535),
 ('flower', 1.9553951154721705),
 ('crazy', 1.9351914543271769),
 ('fall', 1.896361950718139),
 ('worry', 1.889137009235887),
 ('mind', 1.8725577824369803),
 ('shy', 1.8625770823631649),
 ('business', 1.858239331562063),
 ('blah', 1.7916503324920754),
 ('한국', 1.7395776915919268),
 ('yes', 1.6725149283019323),
 ('em', 1.6580317150045465),
 ('control', 1.6275816088166775),
 ('sorry', 1.6181170837152725),
 ('think', 1.6029053675806249),
 ('hit', 1.5946636358702568),
 ('heart', 1.4564988011678022),
 ('down', 1.4448309833777575),
 ('ah', 1.4297295710406557),
 ('ready', 1.4238208197606785),
 ('say', 1.3873513044732313),
 ('matter', 1.3653434369426067),
 ('bad', 1.348649525140968),
 ('only', 1.3440981678289643),
 ('home', 1.3414986661490813),
 ('break', 1.3410441116279574),
 ('tell', 1.3296895882414261),
 ('care

### Topic analysis

In [22]:
from gensim.corpora.dictionary import Dictionary
from gensim.models import LdaModel

In [23]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
def save_lda_vis(lda, corpus, dictionary, filename):
    vis = gensimvis.prepare(lda, corpus, dictionary)
    pyLDAvis.save_html(vis, filename)
    
    with open(filename, "r") as f:
        html = f.read()
        
    return html, vis

In [24]:
def pre_process(df_lyrics):
    # tokenize lyrics
    lyrics_processed = df_lyrics.fillna("").str.split()

    for _list in lyrics_processed:
        for cnt in range(len(_list)):
            _list[cnt] = _list[cnt].replace(".", "").replace(",", "").replace("?", "").replace("\\", "").replace("/", "").replace(":", "").lower()
            
            if is_stopword(_list[cnt]):
                _list[cnt] = ""
                
    # remove empty string
    for cnt in range(len(lyrics_processed)):
        while(True):
            try:
                lyrics_processed[cnt].remove("")
            except:
                break

    return lyrics_processed

#### Among global

In [25]:
df_looking = df[df["grouping"].isin(target_and_global)].reset_index(drop=True)

In [26]:
#train model
lyrics_processed = pre_process(df_looking["lyrics"])
dictionary = Dictionary(lyrics_processed)
corpus = [dictionary.doc2bow(text) for text in lyrics_processed]
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=100)

all topics

In [27]:
lda_html_global, vis = save_lda_vis(lda, corpus, dictionary, "lda.html")

vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
7      0.083584 -0.014373       1        1  3.808373
15    -0.050841 -0.211514       2        1  2.810291
79     0.044554  0.007152       3        1  2.451925
20     0.082324  0.002716       4        1  2.392201
34     0.044520  0.010425       5        1  2.030356
...         ...       ...     ...      ...       ...
97    -0.056629  0.018220      96        1  0.312009
64    -0.188978  0.046481      97        1  0.261766
43    -0.181583 -0.122192      98        1  0.195696
29    -0.115986 -0.000368      99        1  0.170472
23    -0.127095 -0.164720     100        1  0.167048

[100 rows x 5 columns], topic_info=        Term         Freq        Total  Category  logprob  loglift
4853  labour  5070.000000  5070.000000   Default  30.0000  30.0000
88      baby  4918.000000  4918.000000   Default  29.0000  29.0000
862     cake  1822.000000  1822.000000   De

what topics are in kpop?

In [28]:
threshold = 0.3

counter = {}
summer = {}

for cnt in range(len(df_looking)):
    if df_looking["grouping"][cnt] != target_group:
        continue
    
    topics = lda.get_document_topics(corpus[cnt], minimum_probability=0)
    
    for topic in topics:
        if topic[1] > threshold:
            counter[topic[0]] = counter.get(topic[0], 0) + 1
            
        summer[topic[0]] = summer.get(topic[0], 0) + float(topic[1])

In [29]:
_lda_counter_global = sort_dictionary(counter)
lda_counter_global = []
for tup in _lda_counter_global:
    new_tup = [tup[0], tup[1], lda.show_topic(tup[0])]
    lda_counter_global.append(new_tup)
    
lda_counter_global

[[21,
  4,
  [('love', 0.024158427),
   ("can't", 0.017247373),
   ('take', 0.015412984),
   ('need', 0.014465706),
   ('touch', 0.011099063),
   ('wake', 0.0108059775),
   ('let', 0.010398915),
   ("let's", 0.009684799),
   ('tonight', 0.009276307),
   ('g6', 0.0088964645)]],
 [28,
  4,
  [('love', 0.03018291),
   ('worse', 0.025530826),
   ('hot', 0.02088422),
   ('wild', 0.016607676),
   ('power', 0.014125152),
   ('surf', 0.0130786635),
   ('fly', 0.011452182),
   ('baby', 0.009037848),
   ('domino', 0.008319497),
   ('take', 0.007999075)]],
 [18,
  3,
  [('put', 0.018962838),
   ('crazy', 0.018656485),
   ('call', 0.0141171515),
   ("can't", 0.012218787),
   ('way', 0.012078683),
   ('need', 0.011905604),
   ("'em", 0.010363903),
   ('dat', 0.010175418),
   ('down', 0.010125935),
   ('nigger', 0.008698817)]],
 [99,
  3,
  [('tonight', 0.0485636),
   ('let', 0.016475052),
   ('sexy', 0.015490971),
   ('animals', 0.014833955),
   ('say', 0.012355366),
   ('life', 0.010767264),
   ('

In [30]:
_lda_summer_global = sort_dictionary(summer)
lda_summer_global = []
for tup in _lda_summer_global:
    new_tup = [tup[0], tup[1], lda.show_topic(tup[0])]
    lda_summer_global.append(new_tup)
    
lda_summer_global

[[21,
  4.472053622648673,
  [('love', 0.024158427),
   ("can't", 0.017247373),
   ('take', 0.015412984),
   ('need', 0.014465706),
   ('touch', 0.011099063),
   ('wake', 0.0108059775),
   ('let', 0.010398915),
   ("let's", 0.009684799),
   ('tonight', 0.009276307),
   ('g6', 0.0088964645)]],
 [28,
  3.1001946498918187,
  [('love', 0.03018291),
   ('worse', 0.025530826),
   ('hot', 0.02088422),
   ('wild', 0.016607676),
   ('power', 0.014125152),
   ('surf', 0.0130786635),
   ('fly', 0.011452182),
   ('baby', 0.009037848),
   ('domino', 0.008319497),
   ('take', 0.007999075)]],
 [18,
  3.081233017967861,
  [('put', 0.018962838),
   ('crazy', 0.018656485),
   ('call', 0.0141171515),
   ("can't", 0.012218787),
   ('way', 0.012078683),
   ('need', 0.011905604),
   ("'em", 0.010363903),
   ('dat', 0.010175418),
   ('down', 0.010125935),
   ('nigger', 0.008698817)]],
 [99,
  2.4565864380592757,
  [('tonight', 0.0485636),
   ('let', 0.016475052),
   ('sexy', 0.015490971),
   ('animals', 0.01

#### Among KPOP

In [31]:
df_looking = df[df["grouping"].isin(target_and_kpop)].reset_index(drop=True)

In [32]:
#train model
lyrics_processed = pre_process(df_looking["lyrics"])
dictionary = Dictionary(lyrics_processed)
corpus = [dictionary.doc2bow(text) for text in lyrics_processed]
lda = LdaModel(corpus=corpus, id2word=dictionary, num_topics=100)

# observe topics
topics = lda.print_topics()

In [33]:
threshold = 0.3

counter = {}
summer = {}

for cnt in range(len(df_looking)):
    if df_looking["grouping"][cnt] != target_group:
        continue
    
    topics = lda.get_document_topics(corpus[cnt], minimum_probability=0)
    
    for topic in topics:
        if topic[1] > threshold:
            counter[topic[0]] = counter.get(topic[0], 0) + 1
            
        summer[topic[0]] = summer.get(topic[0], 0) + float(topic[1])


In [34]:
lda_html_kpop, vis = save_lda_vis(lda, corpus, dictionary, "lda.html")

vis

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
58    -0.213481 -0.300761       1        1  3.593297
18    -0.053185  0.041583       2        1  2.708714
74    -0.088602  0.051532       3        1  2.573032
16    -0.033154 -0.025504       4        1  2.328964
94    -0.046520  0.048785       5        1  1.927228
...         ...       ...     ...      ...       ...
32     0.050563 -0.046041      96        1  0.398117
24     0.147008 -0.039934      97        1  0.375249
45     0.059878 -0.036191      98        1  0.341991
79     0.152005 -0.104667      99        1  0.306516
50     0.129355 -0.055927     100        1  0.304535

[100 rows x 5 columns], topic_info=         Term         Freq        Total  Category  logprob  loglift
280     child  4079.000000  4079.000000   Default  30.0000  30.0000
425     drunk  1788.000000  1788.000000   Default  29.0000  29.0000
494      cake  1222.000000  1222.000000 

In [35]:
_lda_counter_kpop = sort_dictionary(counter)
lda_counter_kpop = []
for tup in _lda_counter_kpop:
    new_tup = [tup[0], tup[1], lda.show_topic(tup[0])]
    lda_counter_kpop.append(new_tup)
    
lda_counter_kpop

[[16,
  6,
  [('dance', 0.17404926),
   ('dad', 0.03371175),
   ('put', 0.026829407),
   ("'em", 0.013719436),
   ("rollin'", 0.01198159),
   ('sneakers', 0.011114661),
   ('24', 0.010798156),
   ('hours', 0.010496528),
   ('ready', 0.009576908),
   ('lucky', 0.009494716)]],
 [86,
  5,
  [('hot', 0.030475844),
   ('detail', 0.03003586),
   ('bang', 0.014815184),
   ('home', 0.012494936),
   ('love', 0.012239573),
   ('leave', 0.010251263),
   ('let', 0.009898331),
   ("chi't", 0.009440884),
   ('worry', 0.009009873),
   ('bum', 0.008628984)]],
 [37,
  4,
  [('oriente', 0.05101939),
   ('need', 0.02238746),
   ('zu', 0.019168798),
   ('point', 0.016361866),
   ('end', 0.015408443),
   ('spicy', 0.014847646),
   ('jump', 0.011243643),
   ('love', 0.0111910505),
   ('feel', 0.008777883),
   ('rose', 0.00783101)]],
 [50,
  3,
  [('grande', 0.06640082),
   ('shy', 0.02906345),
   ('mr', 0.023206543),
   ('believer', 0.018730633),
   ('sir', 0.016611634),
   ('seguire', 0.012924642),
   ('te

In [36]:
_lda_summer_kpop = sort_dictionary(summer)
lda_summer_kpop = []
for tup in _lda_summer_kpop:
    new_tup = [tup[0], tup[1], lda.show_topic(tup[0])]
    lda_summer_kpop.append(new_tup)
    
lda_summer_kpop

[[16,
  5.731922182100334,
  [('dance', 0.17404926),
   ('dad', 0.03371175),
   ('put', 0.026829407),
   ("'em", 0.013719436),
   ("rollin'", 0.01198159),
   ('sneakers', 0.011114661),
   ('24', 0.010798156),
   ('hours', 0.010496528),
   ('ready', 0.009576908),
   ('lucky', 0.009494716)]],
 [86,
  4.570806341342632,
  [('hot', 0.030475844),
   ('detail', 0.03003586),
   ('bang', 0.014815184),
   ('home', 0.012494936),
   ('love', 0.012239573),
   ('leave', 0.010251263),
   ('let', 0.009898331),
   ("chi't", 0.009440884),
   ('worry', 0.009009873),
   ('bum', 0.008628984)]],
 [37,
  4.170816888944046,
  [('oriente', 0.05101939),
   ('need', 0.02238746),
   ('zu', 0.019168798),
   ('point', 0.016361866),
   ('end', 0.015408443),
   ('spicy', 0.014847646),
   ('jump', 0.011243643),
   ('love', 0.0111910505),
   ('feel', 0.008777883),
   ('rose', 0.00783101)]],
 [50,
  3.1634846217516497,
  [('grande', 0.06640082),
   ('shy', 0.02906345),
   ('mr', 0.023206543),
   ('believer', 0.01873063

### Emotion analysis
Emotions:  
fear
anger
anticipation
trust
surprise
positive
negative
sadness
disgust
joy

In [37]:
import numpy as np

In [38]:
emotions = [
    "anger",
    "anticipation",
    "disgust",
    "fear",
    "joy",
    "negative",	
    "positive",
    "sadness",
    "surprise",	
    "trust",
]

In [39]:
df_nrc = pd.read_csv("NRC.txt", sep="\t", names=["word", "emotion", "amount"])

words_to_emotions = {}

for _, row in df_nrc.iterrows():
    words_to_emotions[row["word"]] = words_to_emotions.get(row["word"], np.zeros((10, 1)))
    words_to_emotions[row["word"]][emotions.index(row["emotion"])] = row["amount"]

In [40]:
df_emotion = df.copy()
for emotion in emotions:
    df_emotion[emotion] = 0

In [41]:
for cnt in range(len(df)):
    lyrics = df["lyrics"][cnt]

    if type(lyrics) != str:
        continue

    emotion_this = np.zeros((10, 1))

    for word in lyrics.split():
        if word in words_to_emotions:
            emotion_this += words_to_emotions[word]
            
    if np.linalg.norm(emotion_this) != 0:
        emotion_this /= np.linalg.norm(emotion_this)
        
    for emotion in emotions:
        df_emotion[emotion][cnt] = emotion_this[emotions.index(emotion)]

C:\Users\Hiyoko\AppData\Local\Temp\ipykernel_9780\3182639529.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_emotion[emotion][cnt] = emotion_this[emotions.index(emotion)]
C:\Users\Hiyoko\AppData\Local\Temp\ipykernel_9780\3182639529.py:17: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[0.06561363]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df_emotion[emotion][cnt] = emotion_this[emotions.index(emotion)]
C:\Users\Hiyoko\AppData\Local\Temp\ipykernel_9780\3182639529.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-

In [42]:
df_emotion_global = df_emotion[df_emotion["grouping"].isin(global_group)]
mean_emotion_global = df_emotion_global[emotions].mean()
normalized_mean_emotion_global = mean_emotion_global / np.linalg.norm(mean_emotion_global)

df_emotion_kpop = df_emotion[df_emotion["grouping"].isin(kpop_group)]
mean_emotion_kpop = df_emotion_kpop[emotions].mean()
normalized_mean_emotion_kpop = mean_emotion_kpop / np.linalg.norm(mean_emotion_kpop)

df_emotion_target = df_emotion[df_emotion["grouping"] == target_group]
mean_emotion_target = df_emotion_target[emotions].mean()
normalized_mean_emotion_target = mean_emotion_target / np.linalg.norm(mean_emotion_target)

In [43]:
normalized_mean_emotion_target

anger           0.316431
anticipation    0.268218
disgust         0.125980
fear            0.235545
joy             0.339116
negative        0.476896
positive        0.518280
sadness         0.222209
surprise        0.140470
trust           0.276452
dtype: float64

against global

In [44]:
emotion_percentage_global = (normalized_mean_emotion_target - normalized_mean_emotion_global) / normalized_mean_emotion_global * 100

emotion_percentage_global

anger           49.766256
anticipation    -9.279161
disgust         -7.783042
fear            -0.174828
joy             -1.692845
negative         2.711875
positive        -3.011005
sadness        -13.678178
surprise       -17.529765
trust           -1.146931
dtype: float64

against KPOP

In [45]:
emotion_percentage_kpop = (normalized_mean_emotion_target - normalized_mean_emotion_kpop) / normalized_mean_emotion_kpop * 100

emotion_percentage_kpop

anger           100.407097
anticipation    -16.543496
disgust          27.199445
fear             13.382179
joy             -12.798583
negative         32.925437
positive        -17.297245
sadness           2.391328
surprise        -15.747100
trust             3.573146
dtype: float64

## Save Data

In [46]:
# to float
for _sum in lda_summer_global:
    _sum[2] = [(tup[0], float(tup[1])) for tup in _sum[2]]
    
for _sum in lda_summer_kpop:
    _sum[2] = [(tup[0], float(tup[1])) for tup in _sum[2]]
    
for count in lda_counter_global:
    count[2] = [(tup[0], float(tup[1])) for tup in count[2]]

for count in lda_counter_kpop:
    count[2] = [(tup[0], float(tup[1])) for tup in count[2]]

In [47]:
json_data = {
    "tfidf_counter_global": tfidf_counter_global,
    "tfidf_summer_global": tfidf_summer_global,
    "tfidf_counter_kpop": tfidf_counter_kpop,
    "tfidf_summer_kpop": tfidf_summer_kpop,
    "lda_counter_global": lda_counter_global,
    "lda_summer_global": lda_summer_global,
    "lda_counter_kpop": lda_counter_kpop,
    "lda_summer_kpop": lda_summer_kpop,
    "lda_html_global": lda_html_global,
    "lda_html_kpop": lda_html_kpop,
    "normalized_mean_emotion_target": normalized_mean_emotion_target.astype(float).tolist(),
    "emotion_percentage_global": emotion_percentage_global.astype(float).tolist(),
    "emotion_percentage_kpop": emotion_percentage_kpop.astype(float).tolist(),
}

In [48]:
import json
with open("data_"+ target_group +".json", "w") as f:
    json.dump(json_data, f)